In [ ]:
from flask import Flask, request, jsonify
from keras.models import load_model
import numpy as np
import joblib
import pandas as pd

In [ ]:
# Membuat instance Flask app
app = Flask(__name__)

# 1. Memuat model Linear Regression dan Scaler untuk Normalisasi
model_lr = load_model('Model/deep_learning_regression_model.h5')  # Model Linear Regression
scaler_lr = joblib.load('Model/scaler_deep_learning_regression.pkl')  # Scaler untuk Linear Regression

# 2. Memuat model Classification dan Scaler untuk Normalisasi
model_class = load_model('Model/classification_model.h5')  # Model Classification
scaler_class = joblib.load('Model/scaler_classification.pkl')  # Scaler untuk Classification

print("Model Linear Regression dan Classification loaded successfully.")

In [ ]:
# Fungsi untuk memprediksi kondisi menggunakan model Classification
def predict_condition(rad_value, input_features):
    """
    Prediksi kondisi (Aman/Tidak Aman) berdasarkan nilai rad(m) yang diprediksi
    dan fitur cuaca lainnya.
    """
    # Menambahkan nilai rad(m) ke input_features untuk model Classification
    input_features['rad_m'] = rad_value

    # Menyiapkan data baru untuk model Classification
    new_data = np.array([[input_features['Tn'], input_features['Tx'], input_features['Tavg'],
                          input_features['RH_avg'], input_features['ff_avg'], input_features['rad_m']]])

    # Melakukan normalisasi data dengan scaler Classification
    new_data_scaled = scaler_class.transform(new_data)

    # Reshape data untuk input model Classification (LSTM)
    new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

    # Menggunakan model Classification untuk prediksi
    prediction = model_class.predict(new_data_reshaped)

    # Menentukan label prediksi (Aman atau Tidak Aman)
    predicted_label = (prediction > 0.5).astype(int)
    return 'Aman' if predicted_label[0][0] == 0 else 'Tidak Aman'


In [ ]:

# Endpoint untuk menerima input dan mengembalikan prediksi
@app.route('/predict', methods=['POST'])
def predict():
    # Mengambil data input dari request body
    data = request.get_json()

    # Memastikan data yang dibutuhkan ada
    required_fields = ['Tn', 'Tx', 'Tavg', 'RH_avg', 'ff_avg']
    if not all(field in data for field in required_fields):
        return jsonify({'error': 'Missing required fields'}), 400

    # Mengambil input features
    input_features = {
        'Tn': data['Tn'],
        'Tx': data['Tx'],
        'Tavg': data['Tavg'],
        'RH_avg': data['RH_avg'],
        'ff_avg': data['ff_avg'],
    }

    # Prediksi nilai rad(m) menggunakan model Linear Regression
    predicted_rad = predict_rad(input_features)

    # Prediksi kondisi berdasarkan rad(m) yang diprediksi
    condition = predict_condition(predicted_rad, input_features)

    # Mengembalikan hasil prediksi dalam format JSON
    return jsonify({
        'predicted_rad': predicted_rad,
        'condition': condition
    })

# Menjalankan aplikasi Flask
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
